# TODO

- [x] IDENTIFICAÇÃO DAS CIDADES LITORÂNEAS
    - [ ] CONSIDERAR CIDADES ESPECÍFICAS
- [x] LEVANTAMENTO DA QUANTIDADE DE HOSPITAIS
    - [x] FILTRAR HOSPITAIS QUE POSSUEM UTI NEONATAL
- [x] LEVANTAMENTO DA SEGURANÇA
    - [x] TAXAS DE HOMICIDIO
    - [x] TAXAS DE ASSALTOS
    - [x] TAXAS DE CRIMES VIOLENTOS
- [x] LEVANTAMENTO DE EDUCAÇÃO
    - [x] INDICE DAS ESCOLAS NO PAÍS
    - [x] QUANTIDADE DE ESCOLAS NA CIDADE
- [ ] LEVANTAMENTO SOBRE LAZER
    - [ ] QTD DE PARQUES
    - [x] QTD DE SHOPPINGS
    - [ ] TOTAL DE ÁREAS VERDES
    - [x] QTD DE PRAIAS
- [ ] LEVANTAMENTO DA MÉDIA DE ALUGUÉIS  
- [x] LEVANTAMENTO DOS PREÇOS DE ALIMENTO
- [ ] DEFINIÇÃO DE CUSTO BENEFÍCIO

## CIDADES A CONSIDERAR

- SP
    - BERTIOGA
    - CANANEIA
    - CARAGUATATUBA
    - GUARUJA
    - IGUAPE
    - ILHABELA
    - ILHA COMPRIDA
    - ITANHAEM
    - MONGAGUA
    - PERUIBE
    - PRAIA GRANDE
    - SANTOS
    - SAO SEBASTIAO
    - SAO VICENTE
    - UBATUBA

- ES
    - VITORIA
    - VILA VELHA

- PB
    - JOAO PESSOA

- SC
    - BALNEARIO CAMBORIU
    - FLORIANOPOLIS

- PR
    - CURITIBA

# IMPORTAÇÃO DE BIBLIOTECAS

In [1]:
import polars as pl
import os
import duckdb

In [2]:
AUXILIAR_PATH = r'finding-home/finding_home/auxiliar/'
DATA_PATH = r'finding-home/finding_home/data/'

# IMPORTAÇÃO DE DADOS

## CIDADES

In [3]:
from unicodedata import normalize 
def custnorm(In_series):
    for i, x in enumerate(In_series):
        newvalue = normalize('NFKD',x).encode('ascii', errors='ignore').decode('utf-8')
        if newvalue != x:
            In_series[i]=newvalue
    return In_series

In [54]:
# cities = pl.read_csv(os.path.join(DATA_PATH, 'CIDADES_LITORANEAS_IBGE.csv'), separator=';') \
#            .filter(pl.col('NM_REGIAO').is_in(['Sul', 'Sudeste', 'Nordeste'])) \
#            .with_columns(pl.col('NM_MUN').str.to_uppercase().alias('MUNICIPIO')) \
#            .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO'))

In [146]:
cities = pl.read_csv(os.path.join(DATA_PATH, 'cidades_ibge_full.csv'), separator='|') \
            .select(pl.col('COD_IBGE'), pl.col('UF'), pl.col('MUNICIPIO'), pl.col('POPULACAO')) \
            .with_columns(pl.col('MUNICIPIO').str.to_uppercase().alias('MUNICIPIO')) \
           .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO'))

In [147]:
cities.head()

COD_IBGE,UF,MUNICIPIO,POPULACAO
i64,str,str,i64
1100015,"""RO""","""ALTA FLORESTA …",22516
1100023,"""RO""","""ARIQUEMES""",111148
1100031,"""RO""","""CABIXI""",5067
1100049,"""RO""","""CACOAL""",86416
1100056,"""RO""","""CEREJEIRAS""",16088


In [148]:
cities.filter(pl.col('MUNICIPIO') == 'OSASCO')

COD_IBGE,UF,MUNICIPIO,POPULACAO
i64,str,str,i64
3534401,"""SP""","""OSASCO""",701428


## HOSPITAIS

In [56]:
hospitals = pl.read_csv(os.path.join(DATA_PATH, 'hospitais.csv'), separator='|', encoding='cp1252', truncate_ragged_lines=True).filter(pl.col('UTI_NEONATAL_EXIST') > 0)

In [57]:
hospitals_stat = duckdb.sql("""
                            SELECT 
                                    REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'HOSPITAL GERAL' THEN 1 ELSE 0 END) AS HOSPITAL_GERAL
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'PRONTO SOCORRO GERAL' THEN 1 ELSE 0 END) AS PRONTO_SOCORRO_GERAL 
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'UNIDADE MISTA' THEN 1 ELSE 0 END) AS UNIDADE_MISTA
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'PRONTO SOCORRO ESPECIALIZADO' THEN 1 ELSE 0 END) AS PRONTO_SOCORRO_ESPECIALIZADO
                                ,   SUM(CASE WHEN DS_TIPO_UNIDADE == 'HOSPITAL ESPECIALIZADO' THEN 1 ELSE 0 END) AS HOSPITAL_ESPECIALIZADO
                            
                                ,  COUNT(DESC_NATUREZA_JURIDICA) AS QTD_HOSPITAIS
                            
                            FROM hospitals

                            GROUP BY REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA

                            ORDER BY REGIAO
                                ,   UF
                                ,   MUNICIPIO
                                ,   DESC_NATUREZA_JURIDICA
                            
                            """).to_df()

In [58]:
duckdb.sql("""SELECT * FROM hospitals_stat WHERE MUNICIPIO = 'OSASCO'""").to_df()

,REGIAO,UF,MUNICIPIO,DESC_NATUREZA_JURIDICA,HOSPITAL_GERAL,PRONTO_SOCORRO_GERAL,UNIDADE_MISTA,PRONTO_SOCORRO_ESPECIALIZADO,HOSPITAL_ESPECIALIZADO,QTD_HOSPITAIS
0,SUDESTE,SP,OSASCO,HOSPITAL_PRIVADO,2.0,0.0,0.0,0.0,0.0,2
1,SUDESTE,SP,OSASCO,HOSPITAL_PÚBLICO,0.0,0.0,0.0,0.0,1.0,1


# ALIMENTOS

In [60]:
food_filter = [
                'ARROZ LONGO FINO BENEFICIADO TIPO 1 (5 kg)'
                , 'FEIJÃO COMUM CORES TIPO 1 (kg)'
                , 'FEIJÃO COMUM PRETO TIPO 1 (kg)'
                , 'CARNE BOVINA ACÉM (kg)'
                , 'CARNE BOVINA COSTELA (kg)'
                , 'CARNE BOVINA PONTA DE AGULHA (kg)'
                , 'CARNE DE FRANGO CONGELADO (kg)'
                , 'CARNE DE FRANGO RESFRIADO (kg)'
                , 'CARNE SUÍNA CONGELADO PERNIL COM OSSO (kg)'
                , 'CARNE SUÍNA RESFRIADO PERNIL COM OSSO (kg)'
                , 'OLEO DE SOJA REFINADO (900 ml)'
                , 'LEITE DE VACA LONGA VIDA INTEGRAL (l)'
                , 'MACARRÃO ESPAGUETE SÊMOLA COM OVOS (500 g)'
                , 'MACARRÃO ESPAGUETE SÊMOLA COM OVOS (kg)'
                , 'SAL (kg)'
                ]

In [61]:
food = pl.read_csv(os.path.join(DATA_PATH, 'ALIMENTOS_CONAB.csv'), separator=';') \
         .melt(id_vars=['UF', 'PRODUTO'], value_vars=pl.selectors.numeric(), variable_name='MES_REF', value_name='PRECO') \
         .filter(pl.col('PRODUTO').is_in(food_filter)) \
         .pivot(values='PRECO', columns='PRODUTO', index='UF', aggregate_function='mean') \
         .fill_null(0.0)
         

In [62]:
food.head()

UF,ARROZ LONGO FINO BENEFICIADO TIPO 1 (5 kg),CARNE BOVINA ACÉM (kg),CARNE BOVINA COSTELA (kg),CARNE BOVINA PONTA DE AGULHA (kg),CARNE DE FRANGO CONGELADO (kg),CARNE DE FRANGO RESFRIADO (kg),CARNE SUÍNA CONGELADO PERNIL COM OSSO (kg),CARNE SUÍNA RESFRIADO PERNIL COM OSSO (kg),FEIJÃO COMUM CORES TIPO 1 (kg),FEIJÃO COMUM PRETO TIPO 1 (kg),LEITE DE VACA LONGA VIDA INTEGRAL (l),MACARRÃO ESPAGUETE SÊMOLA COM OVOS (500 g),MACARRÃO ESPAGUETE SÊMOLA COM OVOS (kg),OLEO DE SOJA REFINADO (900 ml),SAL (kg)
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""ES""",20.27,0.0,22.499091,0.0,10.084545,13.049091,16.122727,0.0,8.373636,6.773636,4.796364,4.995455,0.0,6.657273,2.388182
"""GO""",23.567273,0.0,0.0,17.223636,7.423636,7.917273,0.0,13.353636,8.934545,9.976364,5.564545,0.0,9.150909,6.314545,3.187273
"""MA""",23.606364,0.0,0.0,0.0,0.0,0.0,0.0,14.990909,7.834545,7.345455,5.643636,4.421818,0.0,7.06,1.378182
"""MG""",23.878182,0.0,0.0,22.383636,12.462727,12.236364,0.0,19.38,8.533636,8.107273,4.870909,4.923636,0.0,6.452727,0.0
"""MS""",24.448182,0.0,0.0,16.530909,9.504545,11.877273,0.0,12.293636,9.032727,8.541818,5.535455,0.0,10.160909,6.482727,0.0


# SEGURANÇA

## MORTES

In [68]:
homicidios = pl.read_csv(os.path.join(DATA_PATH, 'taxa-homicidios.csv'), separator=';') \
                  .filter(pl.col('período') > 2013) \
                  .with_columns(pl.col('nome').str.to_uppercase().alias('MUNICIPIO')) \
                  .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO')) \
                  .unique() 

In [69]:
homicidios.head()

cod,nome,período,valor,MUNICIPIO
i64,str,i64,f64,str
5300108,"""Brasília""",2014,30.05,"""BRASILIA"""
5221858,"""Valparaíso de …",2014,55.5,"""VALPARAISO DE …"
5219506,"""Santa Rosa de …",2014,37.43,"""SANTA ROSA DE …"
5218607,"""Rialma""",2014,18.51,"""RIALMA"""
5209903,"""Iaciara""",2014,22.67,"""IACIARA"""


In [45]:
homicidios.filter(pl.col('nome') == 'Osasco').sort('período')

cod,nome,período,valor
i64,str,i64,f64
3534401,"""Osasco""",2014,17.67
3534401,"""Osasco""",2015,18.06
3534401,"""Osasco""",2016,14.85
3534401,"""Osasco""",2017,14.53
3534401,"""Osasco""",2018,13.78
3534401,"""Osasco""",2019,8.59


In [47]:
homicidios.filter(pl.col('nome') == 'Curitiba').sort('período')

cod,nome,período,valor
i64,str,i64,f64
4106902,"""Curitiba""",2014,32.62
4106902,"""Curitiba""",2015,27.72
4106902,"""Curitiba""",2016,29.55
4106902,"""Curitiba""",2017,23.57
4106902,"""Curitiba""",2018,21.18
4106902,"""Curitiba""",2019,16.5


## MORTES ARMA

In [72]:
hom_arma = pl.read_csv(os.path.join(DATA_PATH, 'taxa-de-homicidios-por-armas-de-fogo.csv'), separator=';') \
                  .filter(pl.col('período') > 2013) \
                  .with_columns(pl.col('nome').str.to_uppercase().alias('MUNICIPIO')) \
                  .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO')) \
                  .unique() 

In [73]:
hom_arma.head()

cod,nome,período,valor,MUNICIPIO
i64,str,i64,f64,str
1100023,"""Ariquemes""",2014,46.33,"""ARIQUEMES"""
1100122,"""Ji-Paraná""",2014,12.95,"""JI-PARANA"""
1101476,"""Primavera de R…",2014,0.0,"""PRIMAVERA DE R…"
1101484,"""São Felipe D'O…",2014,0.0,"""SAO FELIPE D'O…"
1101757,"""Vale do Anari""",2014,9.7,"""VALE DO ANARI"""


In [74]:
hom_arma.filter(pl.col('cod') == 3205200 ).sort('período')

cod,nome,período,valor,MUNICIPIO
i64,str,i64,f64,str
3205200,"""Vila Velha""",2014,47.45,"""VILA VELHA"""
3205200,"""Vila Velha""",2015,34.11,"""VILA VELHA"""
3205200,"""Vila Velha""",2016,28.88,"""VILA VELHA"""
3205200,"""Vila Velha""",2017,31.98,"""VILA VELHA"""
3205200,"""Vila Velha""",2018,27.15,"""VILA VELHA"""
3205200,"""Vila Velha""",2019,20.25,"""VILA VELHA"""
3205200,"""Vila Velha""",2020,24.73,"""VILA VELHA"""
3205200,"""Vila Velha""",2021,22.81,"""VILA VELHA"""


In [75]:
hom_arma.filter(pl.col('nome') == 'Osasco' ).sort('período')

cod,nome,período,valor,MUNICIPIO
i64,str,i64,f64,str
3534401,"""Osasco""",2014,11.16,"""OSASCO"""
3534401,"""Osasco""",2015,13.58,"""OSASCO"""
3534401,"""Osasco""",2016,10.81,"""OSASCO"""
3534401,"""Osasco""",2017,11.08,"""OSASCO"""
3534401,"""Osasco""",2018,9.33,"""OSASCO"""
3534401,"""Osasco""",2019,6.3,"""OSASCO"""
3534401,"""Osasco""",2020,7.57,"""OSASCO"""
3534401,"""Osasco""",2021,3.99,"""OSASCO"""


## MORTES VIOLENTAS

In [80]:
m_violentas = pl.read_csv(os.path.join(DATA_PATH, 'mortes-violentas.csv'), separator=';') \
                  .filter(pl.col('período') > 2013) \
                  .with_columns(pl.col('nome').str.to_uppercase().alias('MUNICIPIO')) \
                  .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO')) \
                  .unique() 

In [77]:
m_violentas.head()

cod,nome,período,valor,MUNICIPIO
i64,str,i64,i64,str
5215652,"""Palestina de G…",2014,1,"""PALESTINA DE G…"
5215306,"""Orizona""",2014,13,"""ORIZONA"""
5214903,"""Nova Roma""",2014,5,"""NOVA ROMA"""
5213509,"""Monte Alegre d…",2014,8,"""MONTE ALEGRE D…"
5213103,"""Mineiros""",2014,64,"""MINEIROS"""


In [78]:
m_violentas.filter(pl.col('nome') == 'Osasco' ).sort('período')

cod,nome,período,valor,MUNICIPIO
i64,str,i64,i64,str
3534401,"""Osasco""",2014,419,"""OSASCO"""
3534401,"""Osasco""",2015,383,"""OSASCO"""
3534401,"""Osasco""",2016,361,"""OSASCO"""
3534401,"""Osasco""",2017,408,"""OSASCO"""
3534401,"""Osasco""",2018,376,"""OSASCO"""
3534401,"""Osasco""",2019,326,"""OSASCO"""


## FURTO E ROUBO DE VEÍCULOS

In [84]:
furto_veiculos = pl.read_csv(os.path.join(DATA_PATH, 'furto_roubo_veiculos.csv'), separator=';') \
                    .select(pl.col('UF'), pl.col('ano'), pl.col('qtd'), pl.col('taxa_100mil_veic')) \
                    .unique() 

In [85]:
furto_veiculos.head()

UF,ano,qtd,taxa_100mil_veic
str,i64,i64,f64
"""BA""",2013,17812,568.634017
"""Brasil""",2013,456762,563.280159
"""ES""",2013,7111,451.922179
"""SE""",2014,2801,449.899853
"""PE""",2015,14819,545.694504


## FURTO E ROUBO DE CELULARES

In [89]:
furto_celulares = pl.read_csv(os.path.join(DATA_PATH, 'furto_roubo_celular.csv'), separator=';') \
                    .select(pl.col('UF'), pl.col('ano'), pl.col('qtd'), pl.col('taxa_100mil_hab')) \
                    .unique() 

In [90]:
furto_celulares.head()

UF,ano,qtd,taxa_100mil_hab
str,i64,i64,f64
"""Brasil""",2021,852991,423.722709
"""AM""",2022,40008,1015.128737
"""CE""",2022,41097,467.452894
"""RJ""",2022,46209,287.825413
"""MS""",2021,4350,159.278795


## ESTUPRO

In [87]:
estupro = pl.read_csv(os.path.join(DATA_PATH, 'estupro.csv'), separator=';') \
                    .select(pl.col('UF'), pl.col('ano'), pl.col('qtd'), pl.col('taxa_100mil_hab')) \
                    .unique() 

In [88]:
estupro.head()

UF,ano,qtd,taxa_100mil_hab
str,i64,i64,f64
"""AM""",2009,694,20.451651
"""PE""",2009,1528,17.343423
"""RR""",2009,363,86.121201
"""AC""",2011,225,30.14526
"""PE""",2011,1972,22.245019


# PRAIAS

In [26]:
praias = pl.read_csv(os.path.join(DATA_PATH, 'praias.csv'), separator='|', truncate_ragged_lines=True) \
                    .unique() 

In [28]:
praias.head()

uf,cidade,praia,ponto,latlong,2016,2017,2018,2019,2020,2021,LAT,LONG
str,str,str,str,str,f64,i64,i64,i64,i64,i64,f64,f64
"""AL""","""Coruripe""","""Miai de Cima""",null,"""-10.202028, -3…",3.0,3,3,3,-1,3,-10.202028,-36.189194
"""AL""","""Coruripe""","""Pontal do Coru…",null,"""-10.161306, -3…",3.0,3,3,3,-1,3,-10.161306,-36.13525
"""AL""","""Maceió""","""Praia de Cruz …","""R. Padre Luiz …","""-9.63275, -35.…",1.0,0,0,2,-1,0,-9.63275,-35.696583
"""AL""","""Maceió""","""Praia de Graça…",null,"""-9.583389, -35…",3.0,3,2,3,-1,3,-9.583389,-35.60975
"""AL""","""Maceió""","""Praia de Ipioc…",null,"""-9.531111, -35…",3.0,2,0,3,-1,3,-9.531111,-35.604944


# ESCOLAS

In [35]:
escolas_qtd = pl.read_csv(os.path.join(DATA_PATH, 'qtd_escolas.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [36]:
escolas_qtd.head()

REGIAO,UF,COD_MUN,NM_MUN,ESTADUAL,FEDERAL,MUNICIPAL,PRIVADA
str,str,i64,str,i64,i64,i64,i64
"""Centro-Oeste""","""GO""",5200050,"""Abadia de Goiá…",1,0,5,1
"""Centro-Oeste""","""GO""",5200134,"""Acreúna""",3,0,4,3
"""Centro-Oeste""","""GO""",5201454,"""Aparecida do R…",1,0,1,0
"""Centro-Oeste""","""GO""",5203962,"""Buritinópolis""",1,0,2,0
"""Centro-Oeste""","""GO""",5204656,"""Campinaçu""",1,0,2,0


## ENSINO FUNDAMENTAL 1 - 4

In [29]:
escola_1_4 = pl.read_csv(os.path.join(DATA_PATH, 'escola_1-4.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [30]:
escola_1_4.head()

UF,COD_MUN,NM_MUN,REDE,NOTA
str,i64,str,str,f64
"""RO""",1100015,"""Alta Floresta …","""Pública""",4.8
"""RO""",1100064,"""Colorado do Oe…","""Estadual""",0.0
"""RO""",1100072,"""Corumbiara""","""Municipal""",5.5
"""RO""",1100080,"""Costa Marques""","""Pública""",4.7
"""RO""",1100155,"""Ouro Preto do …","""Estadual""",0.0


## ENSINO FUNDAMENTAL 5 - 8

In [31]:
escola_5_8 = pl.read_csv(os.path.join(DATA_PATH, 'escola_5-8.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [32]:
escola_5_8.head()

UF,COD_MUN,NM_MUN,REDE,NOTA
str,i64,str,str,f64
"""RO""",1100023,"""Ariquemes""","""Estadual""",5.1
"""RO""",1100023,"""Ariquemes""","""Pública""",5.0
"""RO""",1100056,"""Cerejeiras""","""Municipal""",0.0
"""RO""",1100130,"""Machadinho D'O…","""Estadual""",0.0
"""RO""",1100148,"""Nova Brasilând…","""Pública""",5.6


## ENSINO MÉDIO

In [33]:
escola_medio = pl.read_csv(os.path.join(DATA_PATH, 'escola-medio.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [34]:
escola_medio.head()

UF,COD_MUN,NM_MUN,REDE,NOTA
str,i64,str,str,f64
"""RO""",1100072,"""Corumbiara""","""Estadual""",4.4
"""RO""",1100288,"""Rolim de Moura…","""Estadual""",4.6
"""RO""",1100346,"""Alvorada D'Oes…","""Pública""",0.0
"""RO""",1100601,"""Cacaulândia""","""Pública""",4.4
"""RO""",1100924,"""Chupinguaia""","""Estadual""",3.6


# SHOPPINGS

In [37]:
shopping = pl.read_csv(os.path.join(DATA_PATH, 'shoppings_qtd.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [38]:
shopping.head()

UF,QTD
str,i64
"""PI""",5
"""RO""",1
"""MT""",8
"""PA""",10
"""PR""",40


# UNIÃO CIDADES & HOSPITAIS

In [18]:
city_hosp = duckdb.sql("""
           SELECT 
              CT.CD_MUN
            , CT.NM_MUN
            , CT.AREA_KM2
            , CT.CD_UF
            , CT.NM_UF
            , HS.*
            , FD.*

            , HOM.MIN_OBITOS_HOMICIDIOS
            , HOM.MEDIA_OBITOS_HOMICIDIOS
            , HOM.MAX_OBITOS_HOMICIDIOS
            , HOM.QTD_HISTORICO_HOMICIDIOS
            , HOM.ANO_MAIOR_HOMICIDIOS

           FROM hospitals_stat AS HS

           JOIN cities AS CT
           ON CT.SIGLA_UF = HS.UF
           AND CT.MUNICIPIO = HS.MUNICIPIO
           
           JOIN food FD
           ON FD.UF = HS.UF

           JOIN homicidios HOM
           ON HOM.cod = CT.CD_MUN
    
                      
           """).to_df()

In [20]:
city_hosp.head()

,CD_MUN,NM_MUN,AREA_KM2,CD_UF,NM_UF,REGIAO,UF,MUNICIPIO,DESC_NATUREZA_JURIDICA,HOSPITAL_GERAL,...,LEITE DE VACA LONGA VIDA INTEGRAL (l),MACARRÃO ESPAGUETE SÊMOLA COM OVOS (500 g),MACARRÃO ESPAGUETE SÊMOLA COM OVOS (kg),OLEO DE SOJA REFINADO (900 ml),SAL (kg),MIN_OBITOS_HOMICIDIOS,MEDIA_OBITOS_HOMICIDIOS,MAX_OBITOS_HOMICIDIOS,QTD_HISTORICO_HOMICIDIOS,ANO_MAIOR_HOMICIDIOS
0,2702306,Coruripe,897.800,27,Alagoas,NORDESTE,AL,CORURIPE,HOSPITAL_PRIVADO,1.0,...,0.000000,0.00,0.0,7.610000,1.864545,11,16.333333,20,3,2019
1,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_FILANTRÓPICO,4.0,...,0.000000,0.00,0.0,7.610000,1.864545,329,363.666667,400,3,2020
2,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_PRIVADO,12.0,...,0.000000,0.00,0.0,7.610000,1.864545,329,363.666667,400,3,2020
3,2704302,Maceió,509.320,27,Alagoas,NORDESTE,AL,MACEIO,HOSPITAL_PÚBLICO,4.0,...,0.000000,0.00,0.0,7.610000,1.864545,329,363.666667,400,3,2020
4,2900801,Alcobaça,1477.929,29,Bahia,NORDESTE,BA,ALCOBACA,HOSPITAL_PÚBLICO,1.0,...,4.827273,4.15,0.0,6.723636,0.000000,7,15.666667,29,3,2021


In [42]:
duckdb.sql("""SELECT * FROM city_hosp WHERE 
           
           CD_mUN LIKE 'M 3205200' """)

┌─────────┬────────────┬──────────┬───────┬───┬──────────────────────┬──────────────────────┬──────────────────────┐
│ CD_MUN  │   NM_MUN   │ AREA_KM2 │ CD_UF │ … │ MAX_OBITOS_HOMICID…  │ QTD_HISTORICO_HOMI…  │ ANO_MAIOR_HOMICIDIOS │
│  int64  │  varchar   │  double  │ int64 │   │        int64         │        uint32        │        int64         │
├─────────┼────────────┼──────────┼───────┼───┼──────────────────────┼──────────────────────┼──────────────────────┤
│ 3205200 │ Vila Velha │  210.225 │    32 │ … │                  156 │                    3 │                 2020 │
│ 3205200 │ Vila Velha │  210.225 │    32 │ … │                  156 │                    3 │                 2020 │
│ 3205200 │ Vila Velha │  210.225 │    32 │ … │                  156 │                    3 │                 2020 │
├─────────┴────────────┴──────────┴───────┴───┴──────────────────────┴──────────────────────┴──────────────────────┤
│ 3 rows                                                        

In [22]:
duckdb.sql("SELECT * FROM hom_arma WHERE cod = 2702306")

┌─────────┬──────────┬─────────────────────┬───┬─────────────────────┬──────────────────────┬────────────────────┐
│   cod   │   nome   │ MIN_OBITOS_HOM_ARMA │ … │ MAX_OBITOS_HOM_ARMA │ QTD_HISTORICO_HOM_…  │ ANO_MAIOR_HOM_ARMA │
│  int64  │ varchar  │        int64        │   │        int64        │        uint32        │       int64        │
├─────────┼──────────┼─────────────────────┼───┼─────────────────────┼──────────────────────┼────────────────────┤
│ 2702306 │ Coruripe │                   4 │ … │                  20 │                    3 │               2019 │
├─────────┴──────────┴─────────────────────┴───┴─────────────────────┴──────────────────────┴────────────────────┤
│ 1 rows                                                                                     7 columns (6 shown) │
└────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

# LINKS AUXILIARES

- [API IBGE CIDADES](https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[1100015])
- [API IBGE AGREGADA](https://servicodados.ibge.gov.br/api/docs/agregados?versao=3)
- [ALIMENTOS CONAB](https://sisdep.conab.gov.br/precosiagroweb/)
- [ATLAS DA VIOLENCIA](https://www.ipea.gov.br/atlasviolencia/filtros-series)
- [BASE DOS DADOS](https://basedosdados.org/)
- [FORUM SEGURANÇA](http://forumseguranca.org.br:3838/)


In [141]:
# import pandas as pd
# import json

# with open(os.path.join(AUXILIAR_PATH, 'cidades_ibge_full.json'), 'r') as js:
#     ibge = pd.json_normalize(json.load(js), record_path=['resultados', ['series']])
#     ibge.columns = ['COD_IBGE', 'NIVEL_IBGE', 'NIVEL_NOME', 'MUNICIPIO', 'POPULACAO']
#     ibge[['MUNICIPIO', 'UF']] = ibge.MUNICIPIO.str.split(' - ', expand=True)
#     ibge.to_csv(os.path.join(AUXILIAR_PATH, 'cidades_ibge_full.csv'), index=False, sep='|')